In [1]:
import numpy as np

# Metoda Wegierska dla zagadnienia przydzialu ang. Assignment Problem

### Opis zadania

Nalezy wykonac $n$ zadan przy zalozeniu, ze koszt realizacji $i$-tego zadania przez $j$-ta maszyne jest rowny $c_{ij}$. Nalezy znalezc taki przydzial zadan dla mszyn, w ktorym:

- kazda maszyna wykonuje jedno zadanie
- kazde zadanie jest wykonywane przez jedna maszyne
- laczne koszty realizacji sa jak najmniejsze

### Uwagi do metody wegierskiej AP / TSP

- Rozwiazanie problemu nie zmieni sie jesli do dowolnej kolumny lub wiersza macierzy $A$ dodamy/odejmiemy stala wartosc.

- Zbior zer macierzy $A$ nazywa sie niezaleznym, jezeli zadne dwa zera nie leza w jednej kolumnie lub wierszu.

- Jezeli liczba zer niezaleznych jest rowna $n$, to przyjmujac dla odpowiednich im zmiennych $x_{ij}$ wartosci rowne 1, a wszystkie pozostale rowne 0, to otrzymamy rozwiazanie optymalne.

- Wykonanie kroku zwiększania liczby zer niezależnych moze pozostawic ich liczbe bez zmian

- Redukcja macierzy dla zagadnienia AP oraz TSP w zaleznosci od kolejnosci - wiersze/kolumny moze roznic sie sumaryczna wartoscia redukcji oraz moze generowac inny uklad wartosci macierzy zredukowanej

- Algorytm wykreslania zer macierzy minimalna liczba lini kontynuuje zaznaczajac kazdy wiersz majacy w oznakowanej kolumnie zero niezalezne. Zaczyna sie od zaznaczenia kazdego wiersza nie posiadajacego zero niezalezne.

- Minimalna liczba zer niezaleznych po zredukowaniu wynosi 2. Natomiast malsymalna liczba wynosi $n$.

- Algorytm wykreslania zer macierzy minimalna liczba lini wykresla nieoznaczone wiersze i oznaczone kolumny.

### Kwadratowe zagadnienie przydzialu QAP

Przydziela np. zadania do maszyn.


In [102]:
def reduce_matrix(M):
    """Reduces rows and columns"""
    result = M.copy()

    cost = 0
    for row in result:
        row -= min(row)
        cost += min(row)

    result -= np.min(result, axis=0)
    cost += np.sum(np.min(result, axis=0))
    return result, cost

def find_independent_zeros(matrix, tsp=True):
    """Choose independent zeros and fill in dependent with -1"""
    a, b = matrix.shape
    row_free = [1 for _ in range(a)]
    col_free = [1 for _ in range(b)]
    result = matrix.copy()
    pos = []

    for i in range(a):
        for j in range(b):
            if result[i][j] == 0:
                if i == j:
                    result[i][j] = -1
                elif row_free[i] == 1 and col_free[j] == 1:
                    pos.append((i,j))
                    row_free[i] = 0
                    col_free[j] = 0
                else:
                    # Mark position as dependent 0
                    result[i][j] = -1
    return result, pos

def algo_1(matrix, independent_zeros):
    """Finding the minimal number of lines to cover all zeros in the matrix"""
    n, _ = matrix.shape
    independent_rows = [row for row, _ in independent_zeros]
    # independent_cols = []

    # Rows that don't have independent zeros
    marked_rows = [i for i in range(n) if i not in independent_rows]
    # Columns that have dependent zeros for rows_independent
    marked_cols = []
    changed = True

    while changed:
        changed = False
        # Mark columns
        for i in marked_rows:
            for j in range(n):
                if matrix[i][j] == -1 and j not in marked_cols:
                    changed = True
                    marked_cols.append(j)
        
        # Mark rows
        for i in range(n):
            for j in marked_cols:
                if matrix[i][j] == 0 and i not in marked_rows:
                    changed = True
                    marked_rows.append(i)

    return [i for i in range(n) if i not in marked_rows], marked_cols


def convertMatrix(A, marked_rows, marked_cols):
    """Modify matrix to generate more potential results"""
    minimal = np.inf
    n, _ = A.shape
    unmarked_rows = [i for i in range(n) if i not in marked_rows]
    unmarked_cols = [i for i in range(n) if i not in marked_cols]

    # Find minimal element
    for i in unmarked_rows:
        for j in unmarked_cols:
            if A[i][j] > 0 and A[i][j] < minimal:
                minimal = A[i][j]
    
    for i in unmarked_rows:
        A[i,:] -= minimal

    for j in unmarked_cols:
        A[:,j] -= minimal
    
    for i in marked_rows:
        for j in marked_cols:
            A[i][j] += minimal

    A[A < 0] = -1
    return A


def assignment_problem(A, tsp=True):
    n, _ = A.shape
    A, cost = reduce_matrix(A)

    while True:
        A, pos = find_independent_zeros(A, tsp)

        if len(pos) == n:
            return pos
        
        marked_rows, marked_cols = algo_1(A, pos)
        if len(marked_cols) + len(marked_rows) == n:
            # print(marked_cols, marked_rows)
            return pos
        
        A = convertMatrix(A, marked_rows, marked_cols)
    

In [103]:
A = np.array([
    [5, 2, 3, 2, 7],
    [6, 8, 4, 2, 5],
    [6, 4, 3, 7, 2],
    [6, 9, 0, 4, 0],
    [4, 1, 2, 4, 0],
])

B = np.array([
 [-1, -1, 0, -1, 6],
 [0, 5, 1, -1, 4],
 [-1, 0, -1, 3, -1],
 [2, 8, -1, 3, 0],
 [-1, -1, -1, 2, -1]])

pos = [(0, 2), (1, 0), (2, 1), (3, 4)]

assignment_problem(A)
# algo_1(B, pos)

[[-1 -1  0 -1  6]
 [ 0  5  1 -1  4]
 [-1  0 -1  3 -1]
 [ 2  8 -1  3  0]
 [-1 -1 -1  2 -1]]


[(0, 2), (1, 0), (2, 1), (3, 4)]

# Metoda PERT (Program Evaluation and Review Technique)

W metodzie PERT

- Siec o strukturze logicznej zdeterminowanej
- Parametry opisujace poszczegolne czynnosci maja charakter stochastyczny
- Czas trwania kazdej czynnosci jest szacowany na podstawie czasow: optymistycznego, pesymistycznego i najbardziej prawdopodobnego
- Prawdopodobienstwo realizacji na podstawie dystrybuanty rozkladu
- Wariancja calkowita jest wyznaczana jedynie dla czynnosci lezacych na sciezce krytycznej
- Prawdopodobienstwa realizacji przedsiewziecia w terminie wyznaczonym na podstawie czasow oczekiwanych wynosi 1.

Przebieg obliczen dla metody PERT:

- Definiowanie wszystkich czynnosci projektu
- Ustalenie nastepstwa czasowego czynnosci
- Oszacowanie czasu trwania kazdej czynnosci
- Wyznaczenie sciezki krytycznej oraz kryteriow jakosciowych i ilosciowych
- Tworzenie harmonogramu
- Przeszacowanie i poprawki zgodne ze stanem rzeczywistym

# Algorytm Johnsona dla przypadku 2 maszyn

Dana jest macierz $2 \times n$ czasow operacji $t_{ij}$

1. Znajdz najmniejszy element macierzy $t_{ij}$
2. Jesli ten element znajduje sie <br>
- w pierwszym wierszu ($\min t_{ij} = t_{1k}$), to optymalna kolejnosc obroki musi sie rozpoczac detalem o nr k,
- w drugim wierszu ($\min t_{ij} = t_{2s}$), to optymalna kolejnosc konczy sie detalem o numerze s.

|    | Z1 | Z2 | Z3 | Z4 | Z5 | Z6 |
|----|----|----|----|----|----|----|
| M1 | 9  | 6  | 8  | 7  | 12 | 3  |
| M2 | 7  | 3  | 5  | 10 | 4  | 7  |

dla powyzszej tabeli pierwszym zadaniem bedzie Z6 natomiast ostatnim zadaniem bedzie Z2.


# Algorytm Johnsona dla przypadku 3 maszyn

Warunki jakie musza byc spelnione:

- minimalny czas (z wszystkich zadan) dla 1 maszyny $\geq$ maksymalnego czasu dla 2 maszyny

- maksymalny czas (z wszystkich zadan) dla 2 maszyny $\leq$ minimalnego czasu dla 3. maszyny


## Klasyfikacja problemow szeregowania zadan wedlug notacji Grahama

- $\beta$ okresla podzielnosc zadan
- $\alpha$ okresla typ szeregowania: F, J, O
- $\alpha$ okresla liczbe maszyn

# Metoda sciezki krytycznej

### Opis problemu

Podstawa CPM jest stworzenie modelu projektu, ktory zawiera:

- liste wszystkich zadan wymaganych do realizacji projektu
- czas trwania kazdego z zadan
- powiazania pomiedzy poszczegolnymi czynnosciami

CPM pozwala wyliczyc:

- sciezke krytyczna - najdluzsza sciezke dzialan do zakonczenia projektu - ciag czynnosci laczacych zdarzenia o najmniejszych lub zerowych rezerwach czasu.
- najwczesniejszy i najpozniejszy termin wystapienia zdarzenia bez wplywu na dlugosc realizowanego projektu oraz rezerwy czasowe.

Technika ta pozwala na prioretyzacje zadan projektowych poprzez:
- dodanie / podzial zadan, ktore moga byc realizowane rownolegle,
- skrocenie czasu trwania zadan sciezki krytycznej poprzez uzycie dodatkowych zasobow.


### Uwagi do problemu

W metodzie sciezki krytycznej:

- Luz wyznaczamy dla zdarzen.

- Najwczesniejszy mozliwy termin zdarzenia wyznacza maksimum z najwczesniejszego czasu poprzedniego zdarzenia + czas trwania zdarzenia.

<img style="width:350px;height:250px" src="./s1.png">

- Najpozniejszy mozliwy termin zdarzenia wyznacza minimum z roznicy nastepnego zdarzenie oraz bierzacego zdarzenia.

<img style="width:350px;height:250px" src="./s2.png">


# Graph traversal

Zlozonosc obliczeniowa algorytmow przeszukania wynosi $O(|V| + |E|)$

### Depth First Traversals

- Inorder (Left, Root, Right): 4 2 5 1 3
- Preorder (Root, Left, Right): 1 2 4 5 3
- Postorder (Left, Right, Root): 4 5 2 3 1

### Breadth-First or Level Order Traversal

- BFS: 1 2 3 4 5

<img style="width:350px;height:250px" src="./s3.png">

# Teoria grafow

### Terminy

- Najmniejsza liczba krawedzi jaka trzeba usunac z grafu, aby stal sie acykliczny nazywamy liczba cyklometryczna lub rzedem acyklicznosci grafu.

- Macierz ktorej $|N|$ wierszy odpowiada $|N|$ wierzcholkom grafu, a $|E|$ kolumn $|E|$ krawedziom, o elementach 0/1 nazywamy macierza incydencji.

- Podzbior wierzcholkow grafu $G$ taki, ze kazda krawedz $G$ jest incydentna do jakiegos wierzcholka z tego podzbioru, nazywamy pokryciem wierzcholkowym grafu.

- Cykl Eulera w grafie przechodzi przez kazda krawedz dokladnie raz.

- Warunkiem koniecznym i dostatecznym na to, by skonczony graf nieskierowany i spojny posiadal cykl Eulera jest by wszystkie wierzcholki tego grafu mialy rzad parzysty.

- Zagadnienie podzialu jest ogolnym przypadkiem bisekcji grafu.

- Podzbior wierzcholkow grafu $G$ taki ze kazda krawedz $G$ jest incydentna do jakiegos wierzcholka z tego podzbioru, nazywamy pokryciem wierzcholkowym grafu.

- Liczbe drzew rozpinajacych dla grafu mozemy wyznaczyc na podstawie Laplasianu pewnego grafu poprzez wyznaczenie rozwiniecia dla dowolnego elementu macierzy.

- Wierzcholek, po usunieciu ktorego zwieksza sie liczba spojnych skladowych grafu nazywamy przygubem lub wierzcholkiem rozcinajacym.


# Zagadnienie komiwojazera (TSP)

### Opis problemu

Zagadnienie polega na znalezieniu minimalnego cyklu Hamiltona w grafie wazonym.

- Symetryczne zagadnienie komiwojazera polega na tym, ze odleglosc z miasta A do B oraz z miasta B do A jest taka sama.
- Asymetryczne zagadnienie komiwojazera zaklada ze odleglosci miedzy miastami w obie strony nie sa takie same.


### Algorytmy dla TSP

Algorytmy dokladne:

- Przeglad zupelny
- Programowanie dynamiczne
- Metoda podzialu i ograniczen - alg. Little'a. Zabronienie cyklu uwzglednia aktualny zbior odcinkow tworzonego cyklu Hamiltona w podproblemie. W kazdym nowym podproblemie nastepuje wlaczenie jednego odcinka do tworzonego cyklu Hamiltona.
- Alg. Eastmana - relaksacja problemu TSP do AP

Algorytmy przyblizone (zachlanne):

- Alg. najblizszego sasiada (algorytm konstrukcyjny)
- Alg. GTSP, FARIN, NEARIN (algorytm konstrukcyjny)
- Alg. przeszukiwania losowego
- heurystyka Christofidesa (algorytm konstrukcyjny)
- Alg. Ewolucyjne

## Otwarty problem zagadnienia komiwojazera

Optymalne rozwiazanie OTSP poszukujemy w zredukowanej macierzy kosztow.

## TSP metoda wegierska

W metodzie wegierskiej dla zagadnienia komiwojazera dzialamy na macierzy sasiedztwa dla rozpatrywanego grafu, ktorej przekatna jest rowna 0. Celem jest znalezienie minimalnego cyklu Hamiltona. Dla rozpatrywanej macierzy celem jest wyznaczenie $n$ pozycji w macierzy o najmniejszej dlugosci takich, ze w kazdej kolumnie oraz wierszu bedzie dokladnie jedna wybrana pozycja.

- W algorytmie Little'a dla TSP zabronienie podcyklu uwzglednia aktualny zbior odcinkow tworzonego cyklu Hamiltona w podproblemie.



# Programowanie dynamiczne

Metoda programowania dynamicznego moze byc stosowana dla:

- zagadnien statystycznych oraz dynamicznych
- modeli deterministycznych oraz stochastycznych
- zmiennych dyskretnych oraz ciaglych
- skonczonego oraz nieskonczonego horyzontu decyzji

Cechy charakterystyczne:

- wykorzystuje rownanie rekurencyjne w opisie funkcji przejscia

## Problem plecakowy

Dla ponizszego zagadnienia zaladunku:

1. $$8x_1 + 5x_2 + 2x_3 + 6x_4 + 3x_5 -> \max$$
2. $$4x_1 + x_2 + 6x_3 + 6x_4 + 3x_5 \leq 26$$
3. $$0 \leq x_i \leq 6, x_i \in z$$

strategia optymalna musi uwzgledniac ograniczenia (2) w calym procesie decyzyjnym.

1. $$ax_1 + 5x_2 + 2x_3 + 6x_4 + 3x_5 -> \max$$
2. $$bx_1 + x_2 + 6x_3 + 6x_4 + 3x_5 \leq 26$$
3. $$0 \leq x_i \leq 6, x_i \in z$$

W przypadku uwzglednienia przedmiotow o wadze ujemnej $b \leq 0$, wymagane jest rozszerzenie przestrzeni stanow poza zbior [0, 1 .. 26] dla pewnych etapow.

Jesli rownoczesnie $a \leq 0$ i $b \geq 0$ to zmienna $x_1$ mozemy pominac w rozwiazaniu problemu, gdyz $x_1$ bedzie w rozwiazaniu optymalnym zawsze rownym 0.

## Problem podrozy dylizansem

Mieszkaniec wschodniego wybrzeża USA postanowił udać się na zachodnie wybrzeże w czasach, gdy podróże nie należały do przedsięwzięć bezpiecznych.

Agent ubezpieczeniowy przedstawił mu mapę z zaznaczonymi trasami przejazdu dyliżansów i kwotami obowiązujących ubezpieczeń.

Podróżujący postanowił wybrać taką trasę, która charakteryzowałaby się najniższym sumarycznym kosztem ubezpieczenia (i domniemanym maksymalnym bezpieczeństwem).

### Cechy problemu

- Koszt pokonania etapu jest uwzgledniony w funkcji celu.

# Minimalne drzewo rozpinajace

Algorytmy dedykowane do zagadnienia:

- algorytm Boruvki
- algorytm Kruskala
- algorytm Dijkstry-Prima wyznaczania minimalnego drzewa rozpinajacego jest algorytmem konstrukcyjnym. Wykorzystuje on dwie cechy okreslajace: koszt wlaczenia wierzcholka do drzewa i indeks wierzcholka nalezacego do MST.



# Algorytm Farthest Insertion Heuristic (FARIN)

Charakterystka algorytmu:

- sprawdza wszystkie mozliwosci wstawienia wierzcholka w rozwiazaniu czesciowym
- wstawia wierzcholki w kolejnosci od wierzcholka najdalszego

# Algorytmy przeszukiwania najkrotszej sciezki

Algorytmy, ktore wymagaja nieujemnosci wag:

- algorytm Dijkstry

# Metoda podzialu i ograniczen

Warunki jakie sa spelnione w metodzie podzialu i ograniczen dla podzialu problemu $P$ na podproblemy oraz ich relaksacji przy minimalizacji. $RP$ oznacza problem zrelaksowany, $X()$ jest zbiorem rozwiazan, $v(P)$ wartoscia funkcji celu rozwiazania optymalnego problemu.

- $X(P) \in X(RP)$
- $v(P) \geq v(RP)$
- $X(P) = X(P_1) ... X(P_s)$

Kryterium zamykania KZ2 w metodzie podzialu i ograniczen przy maksymalizacji wykorzystuje warunek $v(RP_i) \leq v^*$